In [1]:
import pandas as pd
import psycopg2

import warnings
warnings.filterwarnings("ignore")

In [2]:
energy_raw = pd.read_csv("../data/raw/Electric_Consumption_And_Cost__2010_-_Feb_2023__20240417.csv")
energy_raw.head(3)

,Development Name,Borough,Account Name,Location,Meter AMR,Meter Scope,TDS #,EDP,RC Code,Funding Source,...,Meter Number,Estimated,Current Charges,Rate Class,Bill Analyzed,Consumption (KWH),KWH Charges,Consumption (KW),KW Charges,Other charges
0,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,15396.82,GOV/NYC/068,Yes,128800.0,7387.97,216.0,2808.0,5200.85
1,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,14556.34,GOV/NYC/068,Yes,115200.0,6607.87,224.0,2912.0,5036.47
2,ADAMS,BRONX,ADAMS,BLD 05,NONE,BLD 01 to 07,118.0,248,B011800,FEDERAL,...,7223256,N,13904.98,GOV/NYC/068,Yes,103200.0,5919.55,216.0,2808.0,5177.43


In [3]:
energy_raw.columns

Index(['Development Name', 'Borough', 'Account Name', 'Location', 'Meter AMR',
       'Meter Scope', 'TDS #', 'EDP', 'RC Code', 'Funding Source', 'AMP #',
       'Vendor Name', 'UMIS BILL ID', 'Revenue Month', 'Service Start Date',
       'Service End Date', '# days', 'Meter Number', 'Estimated',
       'Current Charges', 'Rate Class', 'Bill Analyzed', 'Consumption (KWH)',
       'KWH Charges', 'Consumption (KW)', 'KW Charges', 'Other charges'],
      dtype='object')

In [4]:
energy_selected_column =energy_raw[["Borough", "Account Name","Meter Number","Funding Source","Current Charges",\
                                    "TDS #","Revenue Month", "Estimated","Service Start Date", "Service End Date"]]
energy_selected_column.head()

,Borough,Account Name,Meter Number,Funding Source,Current Charges,TDS #,Revenue Month,Estimated,Service Start Date,Service End Date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,118.0,2010-01,N,12/24/09,1/26/10
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,118.0,2010-02,N,1/26/10,2/25/10
2,BRONX,ADAMS,7223256,FEDERAL,13904.98,118.0,2010-03,N,2/25/10,3/26/10
3,BRONX,ADAMS,7223256,FEDERAL,14764.04,118.0,2010-04,N,3/26/10,4/26/10
4,BRONX,ADAMS,7223256,FEDERAL,13729.54,118.0,2010-05,N,4/26/10,5/24/10


In [5]:
# lowercase and replace space with underscore and rename - tds_number
# change columns type

In [6]:
energy_selected_column.dtypes

Borough                object
Account Name           object
Meter Number           object
Funding Source         object
Current Charges       float64
TDS #                 float64
Revenue Month          object
Estimated              object
Service Start Date     object
Service End Date       object
dtype: object

In [7]:
energy_selected_column.rename(columns= 
                                {"Borough":"borough", 
                                "Account Name" :"account_name",
                                "Meter Number":"meter_number",
                                "Funding Source":"funding_source",
                                "Current Charges":"current_charges", 
                                "TDS #":"tds_number",
                                "Revenue Month":"revenue_month",
                                "Estimated":"estimated",
                                "Service Start Date":"service_start_date",
                                "Service End Date":"service_end_date"},inplace = True)

energy_selected_column.head()

,borough,account_name,meter_number,funding_source,current_charges,tds_number,revenue_month,estimated,service_start_date,service_end_date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,118.0,2010-01,N,12/24/09,1/26/10
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,118.0,2010-02,N,1/26/10,2/25/10
2,BRONX,ADAMS,7223256,FEDERAL,13904.98,118.0,2010-03,N,2/25/10,3/26/10
3,BRONX,ADAMS,7223256,FEDERAL,14764.04,118.0,2010-04,N,3/26/10,4/26/10
4,BRONX,ADAMS,7223256,FEDERAL,13729.54,118.0,2010-05,N,4/26/10,5/24/10


In [8]:
energy_selected_column['service_start_date'] = pd.to_datetime(energy_selected_column['service_start_date'])
energy_selected_column['service_end_date'] = pd.to_datetime(energy_selected_column['service_end_date'])


energy_selected_column['revenue_month'] = energy_selected_column['revenue_month'].astype(str) + '-01'
energy_selected_column['revenue_month'] = pd.to_datetime(energy_selected_column['revenue_month'])
#energy_selected_column['revenue_month']  = energy_selected_column['revenue_month'].dt.strftime('%Y-%m-%d')


energy_selected_column['current_charges'] = energy_selected_column ['current_charges'].astype(float)

energy_selected_column.dtypes

borough                       object
account_name                  object
meter_number                  object
funding_source                object
current_charges              float64
tds_number                   float64
revenue_month         datetime64[ns]
estimated                     object
service_start_date    datetime64[ns]
service_end_date      datetime64[ns]
dtype: object

In [9]:
energy_selected_column.head()

,borough,account_name,meter_number,funding_source,current_charges,tds_number,revenue_month,estimated,service_start_date,service_end_date
0,BRONX,ADAMS,7223256,FEDERAL,15396.82,118.0,2010-01-01,N,2009-12-24,2010-01-26
1,BRONX,ADAMS,7223256,FEDERAL,14556.34,118.0,2010-02-01,N,2010-01-26,2010-02-25
2,BRONX,ADAMS,7223256,FEDERAL,13904.98,118.0,2010-03-01,N,2010-02-25,2010-03-26
3,BRONX,ADAMS,7223256,FEDERAL,14764.04,118.0,2010-04-01,N,2010-03-26,2010-04-26
4,BRONX,ADAMS,7223256,FEDERAL,13729.54,118.0,2010-05-01,N,2010-04-26,2010-05-24


In [10]:
energy_40 = energy_selected_column.head(40)

In [11]:
# # Connection parameters
# conn_params = {
#     'dbname': 'nyc_energy',
#     'user': 'postgres',
#     'password': 'postgres',         
#     'host': 'localhost',
#     'port': '5432'
# }

# # Connect to the PostgreSQL database
# conn = psycopg2.connect(**conn_params)
# cur = conn.cursor()

In [12]:
# # Create the 'nyc_energy database
# cur.execute("CREATE DATABASE  nyc_energy")

# # Commit the changes and close the connection to the default database
# conn.commit()
# cur.close()
# conn.close()

In [13]:
energy_40.dtypes

borough                       object
account_name                  object
meter_number                  object
funding_source                object
current_charges              float64
tds_number                   float64
revenue_month         datetime64[ns]
estimated                     object
service_start_date    datetime64[ns]
service_end_date      datetime64[ns]
dtype: object

## load dataframe into postgres


In [14]:
import pandas as pd
import psycopg2
from psycopg2 import extras


 # Connect to PostgreSQL
conn = psycopg2.connect(
    dbname='nyc_energy',
    user='postgres',
    password='postgres',
    host='localhost',
    port='5432'
)

# Create a cursor for database operations
cur = conn.cursor()

# Define the PostgreSQL table structure (if needed)
create_table_query = """
CREATE TABLE IF NOT EXISTS raw_energy_data (
    id SERIAL PRIMARY KEY,
    borough VARCHAR(100),
    account_name VARCHAR(100),
    funding_source VARCHAR(100), 
    meter_number INT,
    revenue_month DATE,
    service_end_date DATE,
    service_start_date DATE,
    tds_number FLOAT,
    current_charges FLOAT
)
"""
cur.execute(create_table_query)

# Define the insertion query for bulk insertion
insert_query = """
INSERT INTO raw_energy_data (borough, account_name, funding_source, meter_number, revenue_month, service_end_date, service_start_date, tds_number, current_charges) VALUES %s
"""

# Define the insertion query for bulk insertion
values = [tuple(row) for row in energy_40[['borough', 'account_name', 'funding_source', 'meter_number', 'revenue_month', 'service_end_date', 'service_start_date', 'tds_number', 'current_charges']].values]

# Execute the insertion query with corrected values
extras.execute_values(cur, insert_query, values)

# Commit changes
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

print("Data inserted successfully.")

Data inserted successfully.


In [15]:
energy_selected_column[['borough', 'meter_number']].values

array([['BRONX', '7223256'],
       ['BRONX', '7223256'],
       ['BRONX', '7223256'],
       ...,
       ['BROOKLYN', '2786583'],
       ['BROOKLYN', '2786584'],
       ['BROOKLYN', '2786586']], dtype=object)

In [16]:
[tuple(row) for row in energy_selected_column[['borough', 'meter_number']].head(10).values]

[('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256'),
 ('BRONX', '7223256')]